In [211]:
import pandas as pd
import numpy as np

In [212]:
df = pd.read_parquet('rail_data_cleaned_20230728132300.parquet')

### Dropping columns we don't need

In [213]:
dropped = ['working_time_pass', 'pass', 'train_length']

In [214]:
df = df.drop(columns=dropped)

## Round 1
#### Eliminating rows with duplicate id and train platforms in conjunction with null actuals

In [215]:
# Subset of columns to consider for duplicates
subset_columns = ['unique_id', 'train_platform']

# Find rows with duplicated unique_id and train_platform
duplicates = df.duplicated(subset=subset_columns, keep=False)

# Find rows with null values in actual_arrival_time and actual_departure_time
null_rows = (df['actual_arrival_time'].isnull()) & (df['actual_departure_time'].isnull())

# Combine the conditions to get the rows to be removed
rows_to_remove = duplicates & null_rows

# Keep the rows that are not in rows_to_remove
df_cleaned = df[~rows_to_remove]


#### Looking at a busy route to evaluate cleaning

In [216]:
popular_id = df_cleaned[df_cleaned['unique_id'] == 'Y55129']

In [217]:
popular_id

,route_id,unique_id,service_start_date,update_origin,train_platform,working_time_arrival,working_time_departure,planned_time_arrival,planned_time_departure,platform,actual_arrival_time,actual_departure_time,is_delayed_arrival,is_delayed_departure
24918,202307288955129,Y55129,2023-07-28,Trust,BRENTX,2023-07-28 01:13:30,2023-07-28 01:14:00,NaT,NaT,1,2023-07-28 01:19:00,2023-07-28 01:20:00,False,False
24919,202307288955129,Y55129,2023-07-28,Trust,CRKLWD,2023-07-28 01:15:30,2023-07-28 01:17:00,2023-07-28 01:16:00,2023-07-28 01:17:00,1,2023-07-28 01:21:00,2023-07-28 01:22:00,False,False
24920,202307288955129,Y55129,2023-07-28,Trust,WHMPSTM,2023-07-28 01:20:00,2023-07-28 01:20:30,2023-07-28 01:20:00,2023-07-28 01:20:00,1,2023-07-28 01:24:00,2023-07-28 01:25:00,False,False
24922,202307288955129,Y55129,2023-07-28,Trust,KNTSHTN,2023-07-28 01:24:00,2023-07-28 01:24:30,2023-07-28 01:24:00,2023-07-28 01:24:00,1,2023-07-28 01:28:00,2023-07-28 01:29:00,False,False
24924,202307288955129,Y55129,2023-07-28,Trust,STPXBOX,2023-07-28 01:30:00,2023-07-28 01:32:00,2023-07-28 01:30:00,2023-07-28 01:32:00,A,2023-07-28 01:33:00,2023-07-28 01:34:00,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92756,202307288955129,Y55129,2023-07-28,Darwin,SELHRST,2023-07-28 02:02:30,2023-07-28 02:03:00,NaT,NaT,1,2023-07-28 02:04:00,2023-07-28 02:04:00,False,False
92758,202307288955129,Y55129,2023-07-28,Darwin,ECROYDN,2023-07-28 02:06:00,2023-07-28 02:07:00,2023-07-28 02:06:00,2023-07-28 02:07:00,5,2023-07-28 02:06:00,2023-07-28 02:07:00,False,False
92761,202307288955129,Y55129,2023-07-28,Darwin,PURLEY,2023-07-28 02:12:00,2023-07-28 02:13:00,2023-07-28 02:12:00,2023-07-28 02:13:00,2,2023-07-28 02:12:00,2023-07-28 02:13:00,False,False
92768,202307288955129,Y55129,2023-07-28,Darwin,HORLEY,2023-07-28 02:25:30,2023-07-28 02:26:00,2023-07-28 02:26:00,2023-07-28 02:26:00,2,2023-07-28 02:26:00,2023-07-28 02:26:00,False,False


In [218]:
# Looking at the number of unique train platforms

popular_id['train_platform'].unique()

array(['BRENTX', 'CRKLWD', 'WHMPSTM', 'KNTSHTN', 'STPXBOX', 'FRNDNLT',
       'BLFR', 'SELHRST', 'ECROYDN', 'PURLEY', 'HORLEY', 'GTWK',
       'LEAGRVE', 'LUTON', 'LUTOAPY', 'HRPNDN', 'STALBCY', 'RADLETT',
       'ELTR', 'MLHB', 'HDON', 'FLITWCK', 'HRLG'], dtype=object)

In [219]:
popular_id['train_platform'].nunique()

23

## Round 2
### Keeping rows with Darwin and Trust

In [220]:
# Find duplicate rows with specific update_origin values
duplicates_to_remove = df_cleaned[(df_cleaned.duplicated(['unique_id', 'train_platform'], keep=False))
                          & (df_cleaned['update_origin'].isin(['TD', 'Trust', 'Darwin']))]

# Drop rows with update_origin as 'TD'
df_filtered = df_cleaned.drop(duplicates_to_remove[duplicates_to_remove['update_origin'] == 'TD'].index)


#### Looking at a busy route to evaluate cleaning

In [221]:
popular_id = df_filtered[df_filtered['unique_id'] == 'Y55129']

In [222]:
popular_id

,route_id,unique_id,service_start_date,update_origin,train_platform,working_time_arrival,working_time_departure,planned_time_arrival,planned_time_departure,platform,actual_arrival_time,actual_departure_time,is_delayed_arrival,is_delayed_departure
24918,202307288955129,Y55129,2023-07-28,Trust,BRENTX,2023-07-28 01:13:30,2023-07-28 01:14:00,NaT,NaT,1,2023-07-28 01:19:00,2023-07-28 01:20:00,False,False
24919,202307288955129,Y55129,2023-07-28,Trust,CRKLWD,2023-07-28 01:15:30,2023-07-28 01:17:00,2023-07-28 01:16:00,2023-07-28 01:17:00,1,2023-07-28 01:21:00,2023-07-28 01:22:00,False,False
24920,202307288955129,Y55129,2023-07-28,Trust,WHMPSTM,2023-07-28 01:20:00,2023-07-28 01:20:30,2023-07-28 01:20:00,2023-07-28 01:20:00,1,2023-07-28 01:24:00,2023-07-28 01:25:00,False,False
24922,202307288955129,Y55129,2023-07-28,Trust,KNTSHTN,2023-07-28 01:24:00,2023-07-28 01:24:30,2023-07-28 01:24:00,2023-07-28 01:24:00,1,2023-07-28 01:28:00,2023-07-28 01:29:00,False,False
24924,202307288955129,Y55129,2023-07-28,Trust,STPXBOX,2023-07-28 01:30:00,2023-07-28 01:32:00,2023-07-28 01:30:00,2023-07-28 01:32:00,A,2023-07-28 01:33:00,2023-07-28 01:34:00,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92756,202307288955129,Y55129,2023-07-28,Darwin,SELHRST,2023-07-28 02:02:30,2023-07-28 02:03:00,NaT,NaT,1,2023-07-28 02:04:00,2023-07-28 02:04:00,False,False
92758,202307288955129,Y55129,2023-07-28,Darwin,ECROYDN,2023-07-28 02:06:00,2023-07-28 02:07:00,2023-07-28 02:06:00,2023-07-28 02:07:00,5,2023-07-28 02:06:00,2023-07-28 02:07:00,False,False
92761,202307288955129,Y55129,2023-07-28,Darwin,PURLEY,2023-07-28 02:12:00,2023-07-28 02:13:00,2023-07-28 02:12:00,2023-07-28 02:13:00,2,2023-07-28 02:12:00,2023-07-28 02:13:00,False,False
92768,202307288955129,Y55129,2023-07-28,Darwin,HORLEY,2023-07-28 02:25:30,2023-07-28 02:26:00,2023-07-28 02:26:00,2023-07-28 02:26:00,2,2023-07-28 02:26:00,2023-07-28 02:26:00,False,False


In [223]:
# Looking at the number of unique train platforms

popular_id['train_platform'].unique()

array(['BRENTX', 'CRKLWD', 'WHMPSTM', 'KNTSHTN', 'STPXBOX', 'FRNDNLT',
       'BLFR', 'SELHRST', 'ECROYDN', 'PURLEY', 'HORLEY', 'GTWK',
       'LEAGRVE', 'LUTON', 'LUTOAPY', 'HRPNDN', 'STALBCY', 'RADLETT',
       'ELTR', 'MLHB', 'HDON', 'FLITWCK', 'HRLG'], dtype=object)

In [224]:
popular_id['train_platform'].nunique()

23

## Round 3
### Keeping rows with Darwin and Trust

In [225]:
# Find duplicate rows with specific update_origin values
duplicates_to_remove = df_filtered[(df_filtered.duplicated(['unique_id', 'train_platform'], keep=False))
                                  & (df_filtered['update_origin'].isin(['CIS', 'Trust', 'Darwin']))]

# Drop rows with update_origin as 'TD'
new_df = df_filtered.drop(duplicates_to_remove[duplicates_to_remove['update_origin'] == 'Trust'].index)

In [226]:
popular_id = new_df[new_df['unique_id'] == 'Y55129']

In [227]:
popular_id

,route_id,unique_id,service_start_date,update_origin,train_platform,working_time_arrival,working_time_departure,planned_time_arrival,planned_time_departure,platform,actual_arrival_time,actual_departure_time,is_delayed_arrival,is_delayed_departure
34022,202307288955129,Y55129,2023-07-28,Darwin,LEAGRVE,2023-07-28 00:35:30,2023-07-28 00:36:00,2023-07-28 00:36:00,2023-07-28 00:36:00,1,NaT,2023-07-28 00:41:00,False,False
34023,202307288955129,Y55129,2023-07-28,Darwin,LUTON,2023-07-28 00:39:00,2023-07-28 00:40:30,2023-07-28 00:39:00,2023-07-28 00:40:00,1,2023-07-28 00:44:00,2023-07-28 00:45:00,False,False
34024,202307288955129,Y55129,2023-07-28,Darwin,LUTOAPY,2023-07-28 00:42:30,2023-07-28 00:43:30,2023-07-28 00:43:00,2023-07-28 00:43:00,1,2023-07-28 00:47:00,2023-07-28 00:47:00,False,False
34025,202307288955129,Y55129,2023-07-28,Darwin,HRPNDN,2023-07-28 00:48:30,2023-07-28 00:49:00,2023-07-28 00:49:00,2023-07-28 00:49:00,1,2023-07-28 00:52:00,2023-07-28 00:53:00,False,False
34027,202307288955129,Y55129,2023-07-28,Darwin,STALBCY,2023-07-28 00:54:30,2023-07-28 00:55:30,2023-07-28 00:55:00,2023-07-28 00:55:00,1,2023-07-28 00:57:00,2023-07-28 00:58:00,False,False
34028,202307288955129,Y55129,2023-07-28,Darwin,RADLETT,2023-07-28 01:00:30,2023-07-28 01:01:00,2023-07-28 01:01:00,2023-07-28 01:01:00,1,2023-07-28 01:03:00,2023-07-28 01:03:00,False,False
34030,202307288955129,Y55129,2023-07-28,Darwin,ELTR,2023-07-28 01:04:00,2023-07-28 01:04:30,2023-07-28 01:04:00,2023-07-28 01:04:00,1,2023-07-28 01:06:00,2023-07-28 01:07:00,False,False
34031,202307288955129,Y55129,2023-07-28,Darwin,MLHB,2023-07-28 01:07:30,2023-07-28 01:08:30,2023-07-28 01:08:00,2023-07-28 01:08:00,1,2023-07-28 01:10:00,2023-07-28 01:10:00,False,False
34033,202307288955129,Y55129,2023-07-28,Darwin,HDON,2023-07-28 01:11:30,2023-07-28 01:12:00,2023-07-28 01:12:00,2023-07-28 01:12:00,1,2023-07-28 01:12:00,2023-07-28 01:14:00,False,False
34034,202307288955129,Y55129,2023-07-28,Darwin,BRENTX,2023-07-28 01:13:30,2023-07-28 01:14:00,NaT,NaT,1,2023-07-28 01:15:00,2023-07-28 01:16:00,False,False


In [228]:
popular_id['train_platform'].unique()

array(['LEAGRVE', 'LUTON', 'LUTOAPY', 'HRPNDN', 'STALBCY', 'RADLETT',
       'ELTR', 'MLHB', 'HDON', 'BRENTX', 'CRKLWD', 'WHMPSTM', 'KNTSHTN',
       'STPXBOX', 'FLITWCK', 'HRLG', 'ECROYDN', 'PURLEY', 'HORLEY',
       'GTWK', 'FRNDNLT', 'BLFR', 'SELHRST'], dtype=object)

## Round 4
### Keeping rows with Darwin

In [229]:
# Find duplicate rows with specific update_origin values
duplicates_to_remove = new_df[(new_df.duplicated(['unique_id', 'train_platform'], keep=False))
                                   & (new_df['update_origin'].isin(['CIS', 'Darwin']))]

# Drop rows with update_origin as 'TD'
df4 = new_df.drop(duplicates_to_remove[duplicates_to_remove['update_origin'] == 'CIS'].index)

In [230]:
popular_id = df4[df4['unique_id'] == 'Y55129']

In [231]:
popular_id

,route_id,unique_id,service_start_date,update_origin,train_platform,working_time_arrival,working_time_departure,planned_time_arrival,planned_time_departure,platform,actual_arrival_time,actual_departure_time,is_delayed_arrival,is_delayed_departure
34022,202307288955129,Y55129,2023-07-28,Darwin,LEAGRVE,2023-07-28 00:35:30,2023-07-28 00:36:00,2023-07-28 00:36:00,2023-07-28 00:36:00,1,NaT,2023-07-28 00:41:00,False,False
34023,202307288955129,Y55129,2023-07-28,Darwin,LUTON,2023-07-28 00:39:00,2023-07-28 00:40:30,2023-07-28 00:39:00,2023-07-28 00:40:00,1,2023-07-28 00:44:00,2023-07-28 00:45:00,False,False
34024,202307288955129,Y55129,2023-07-28,Darwin,LUTOAPY,2023-07-28 00:42:30,2023-07-28 00:43:30,2023-07-28 00:43:00,2023-07-28 00:43:00,1,2023-07-28 00:47:00,2023-07-28 00:47:00,False,False
34025,202307288955129,Y55129,2023-07-28,Darwin,HRPNDN,2023-07-28 00:48:30,2023-07-28 00:49:00,2023-07-28 00:49:00,2023-07-28 00:49:00,1,2023-07-28 00:52:00,2023-07-28 00:53:00,False,False
34027,202307288955129,Y55129,2023-07-28,Darwin,STALBCY,2023-07-28 00:54:30,2023-07-28 00:55:30,2023-07-28 00:55:00,2023-07-28 00:55:00,1,2023-07-28 00:57:00,2023-07-28 00:58:00,False,False
34028,202307288955129,Y55129,2023-07-28,Darwin,RADLETT,2023-07-28 01:00:30,2023-07-28 01:01:00,2023-07-28 01:01:00,2023-07-28 01:01:00,1,2023-07-28 01:03:00,2023-07-28 01:03:00,False,False
34030,202307288955129,Y55129,2023-07-28,Darwin,ELTR,2023-07-28 01:04:00,2023-07-28 01:04:30,2023-07-28 01:04:00,2023-07-28 01:04:00,1,2023-07-28 01:06:00,2023-07-28 01:07:00,False,False
34031,202307288955129,Y55129,2023-07-28,Darwin,MLHB,2023-07-28 01:07:30,2023-07-28 01:08:30,2023-07-28 01:08:00,2023-07-28 01:08:00,1,2023-07-28 01:10:00,2023-07-28 01:10:00,False,False
34033,202307288955129,Y55129,2023-07-28,Darwin,HDON,2023-07-28 01:11:30,2023-07-28 01:12:00,2023-07-28 01:12:00,2023-07-28 01:12:00,1,2023-07-28 01:12:00,2023-07-28 01:14:00,False,False
34034,202307288955129,Y55129,2023-07-28,Darwin,BRENTX,2023-07-28 01:13:30,2023-07-28 01:14:00,NaT,NaT,1,2023-07-28 01:15:00,2023-07-28 01:16:00,False,False


In [232]:
popular_id['train_platform'].unique()

array(['LEAGRVE', 'LUTON', 'LUTOAPY', 'HRPNDN', 'STALBCY', 'RADLETT',
       'ELTR', 'MLHB', 'HDON', 'BRENTX', 'CRKLWD', 'WHMPSTM', 'KNTSHTN',
       'STPXBOX', 'FLITWCK', 'HRLG', 'FRNDNLT', 'BLFR', 'SELHRST',
       'ECROYDN', 'PURLEY', 'HORLEY', 'GTWK'], dtype=object)

### Now that we have cleaned update origin we need to keep rows with the best actual time data

In [233]:
# Group by 'unique_id' and 'train_platform', and aggregate using min and max for desired columns
agg_functions = {
    'actual_arrival_time': 'min',
    'actual_departure_time': 'max'
}

aggregated_df = df4.groupby(['unique_id', 'train_platform'], as_index=False).agg(agg_functions)


In [234]:
aggregated_df

,unique_id,train_platform,actual_arrival_time,actual_departure_time,update_origin,working_time_arrival,working_time_departure
0,C00066,LINCLNC,2023-07-28 05:47:00,2023-07-28 05:48:00,Darwin,2023-07-28 05:46:30,2023-07-28 05:48:30
1,C00066,LOWDHAM,2023-07-28 05:11:00,2023-07-28 05:12:00,Darwin,2023-07-28 05:11:00,2023-07-28 05:12:00
2,C00066,NWRKCAS,2023-07-28 05:24:00,2023-07-28 05:24:00,Darwin,2023-07-28 05:23:30,2023-07-28 05:24:30
3,C00069,SILEBY,NaT,2023-07-27 18:07:00,TD,2023-07-27 18:01:30,2023-07-27 18:02:30
4,C00071,CLHM,NaT,NaT,Trust,2023-07-27 20:13:00,2023-07-27 20:13:30
...,...,...,...,...,...,...,...
24778,Y55367,WORTHNG,2023-07-28 06:28:00,2023-07-28 06:28:00,CIS,2023-07-28 06:27:30,2023-07-28 06:28:30
24779,Y55367,WVLSFLD,2023-07-28 07:00:00,2023-07-28 07:01:00,CIS,2023-07-28 07:00:00,2023-07-28 07:01:00
24780,Y55367,WWRTHNG,2023-07-28 06:25:00,2023-07-28 06:25:00,CIS,2023-07-28 06:24:30,2023-07-28 06:25:30
24781,Y55368,ECROYDN,2023-07-27 17:47:00,2023-07-27 17:47:00,TD,2023-07-27 17:46:30,2023-07-27 17:47:30


In [235]:
aggregated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   unique_id               24783 non-null  object        
 1   train_platform          24783 non-null  object        
 2   actual_arrival_time     20654 non-null  datetime64[ns]
 3   actual_departure_time   22719 non-null  datetime64[ns]
 4   update_origin           24325 non-null  object        
 5   working_time_arrival    24783 non-null  datetime64[ns]
 6   working_time_departure  24783 non-null  datetime64[ns]
dtypes: datetime64[ns](4), object(3)
memory usage: 1.3+ MB


In [236]:
popular_id = aggregated_df[aggregated_df['unique_id'] == 'Y55129']

In [237]:
popular_id

,unique_id,train_platform,actual_arrival_time,actual_departure_time,update_origin,working_time_arrival,working_time_departure
24498,Y55129,BLFR,2023-07-28 01:39:00,2023-07-28 01:43:00,Darwin,2023-07-28 01:39:00,2023-07-28 01:40:00
24499,Y55129,BRENTX,2023-07-28 01:15:00,2023-07-28 01:21:00,Darwin,2023-07-28 01:13:30,2023-07-28 01:14:00
24500,Y55129,CRKLWD,2023-07-28 01:16:00,2023-07-28 01:23:00,Darwin,2023-07-28 01:15:30,2023-07-28 01:17:00
24501,Y55129,ECROYDN,2023-07-28 02:06:00,2023-07-28 02:10:00,Darwin,2023-07-28 02:06:00,2023-07-28 02:07:00
24502,Y55129,ELTR,2023-07-28 01:06:00,2023-07-28 01:10:00,Darwin,2023-07-28 01:04:00,2023-07-28 01:04:30
24503,Y55129,FLITWCK,2023-07-28 00:27:00,2023-07-28 00:27:00,TD,2023-07-28 00:25:30,2023-07-28 00:26:00
24504,Y55129,FRNDNLT,2023-07-28 01:37:00,2023-07-28 01:40:00,Darwin,2023-07-28 01:35:30,2023-07-28 01:36:30
24505,Y55129,GTWK,2023-07-28 02:30:00,2023-07-28 02:34:00,Darwin,2023-07-28 02:29:30,2023-07-28 02:31:30
24506,Y55129,HDON,2023-07-28 01:12:00,2023-07-28 01:19:00,Darwin,2023-07-28 01:11:30,2023-07-28 01:12:00
24507,Y55129,HORLEY,2023-07-28 02:26:00,2023-07-28 02:29:00,Darwin,2023-07-28 02:25:30,2023-07-28 02:26:00
